A Multimodal Interactive-Agent Framework for Inclusive and Personalized Early Dementia Assessment Using Online and Offline Cognitive Tasks
Paper:
1) Digital healthcare Application: multimodal agent for early dementia assessent and improvement
2) Playing games require mental concentration, memory and quick motor reaction in simulating brain to work and gain memory back.
3) Findings show the impacts of game-based intervention in terms of cognitive improvement, improved memory recall proven through the improved time in assembling the puzzle, and improved skills and IT literacy among elderly

0. Include healthy users + those with MCI (mild cognitive impairment), or at risk (family history)
1. LLM-based semantic scoring:
2. Dynamically generates personalized suggestions



In [ ]:
# --- STEP 1: Install Required Packages ---
!pip install transformers sentence-transformers nltk accelerate --quiet



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.4 MB/s eta 0:00:00


In [ ]:
# --- STEP 2: Import Libraries ---
import datetime, time, random, nltk
from nltk.corpus import wordnet as wn
from sentence_transformers import SentenceTransformer, util
import torch
#import speech_recognition as sr

nltk.download('wordnet')
nltk.download('omw-1.4')



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
# --- STEP 3: Initialize Model ---
encoder = SentenceTransformer('all-MiniLM-L6-v2')
today_weekday = datetime.datetime.today().strftime('%A')

# --- STEP 5: Animal Checker ---
# --- STEP 4: Animal Checker ---
def is_animal(word):
    word = word.strip().lower()
    synsets = wn.synsets(word, pos=wn.NOUN)
    for syn in synsets[:2]:
        for path in syn.hypernym_paths():
            for h in path:
                if 'animal.n.01' in h.name():
                    return True
    return False

def score_animals(user_input):
    words = [w.strip().lower() for w in user_input.split(",")]
    matched_animals = [w for w in words if is_animal(w)]
    return round(min(len(matched_animals), 3) / 3, 2)

# --- STEP 5: Number Recall Game ---
def number_recall_game():
    digit_length = random.choice([4, 5, 6])
    number = ''.join([str(random.randint(0, 9)) for _ in range(digit_length)])
    print(f"\nREMEMBER THIS NUMBER: {number}")
    time.sleep(5)
    print("\n" * 50)
    user_input = input("Now type the number you saw: ")
    correct_digits = sum([1 for a, b in zip(number, user_input) if a == b])
    score = round(correct_digits / digit_length, 2)
    return score, number, user_input

def word_list_recall_game():
    print("\n🧠 Word Recall Game")
    word_list = ["apple", "table", "car", "banana", "house"]
    print("Memorize these words:")
    print(", ".join(word_list))
    time.sleep(5)
    print("\n" * 50)

    print("Now, please recall and type as many words as you remember (separated by commas):")
    user_input = input("Your answer: ").lower()
    user_words = [w.strip() for w in user_input.split(",")]

    correct = sum([1 for word in word_list if word in user_words])
    score = round(correct / len(word_list), 2)
    return score, ", ".join(word_list), ", ".join(user_words)




In [ ]:
# --- STEP 7: Run the Full Test ---
questions = [
    "1. What day of the week is it?",
    "2. Can you name three animals?",
    "3. Please repeat this sentence: The cat sat on the mat."
]

expected_answers = [
    today_weekday,
    "",
    "The cat sat on the mat"
]

score_details = []

print("\n🧠 Welcome to the Cognitive Check")

for i in range(len(questions)):
    print("\n" + questions[i])
    user_input = input("Your answer: ")

    if i == 1:
        score = score_animals(user_input)
    else:
        user_vec = encoder.encode(user_input, convert_to_tensor=True)
        expected_vec = encoder.encode(expected_answers[i], convert_to_tensor=True)
        score = float(util.pytorch_cos_sim(user_vec, expected_vec)[0])

    score_details.append({
        "type": "text",
        "question": questions[i],
        "expected": expected_answers[i] if expected_answers[i] else "Any 3 animals",
        "user_answer": user_input,
        "score": round(score, 2)
    })

  # --- Number Recall ---
recall_score, shown_number, recalled_number = number_recall_game()
score_details.append({
    "type": "memory",
    "question": "4. Number Recall Game",
    "expected": shown_number,
    "user_answer": recalled_number,
    "score": round(recall_score, 2)
})


# --- Memory Game ---
recall_score, original_words, recalled_words = word_list_recall_game()
score_details.append({
    "type": "recall",
    "question": "5. Word List Recall Game",
    "expected": original_words,
    "user_answer": recalled_words,
    "score": round(recall_score, 2)
})



# --- Final Score ---
final_score = round(sum([d['score'] for d in score_details]) / len(score_details), 2)

print("\n🧾 Detailed Scoring Summary:\n")
for d in score_details:
    print(f"{d['question']}")
    print(f"Expected: {d['expected']}")
    print(f"Your Answer: {d['user_answer']}")
    print(f"Score: {d['score']} / 1.0\n")

print(f"🧠 Final Combined Cognitive Score: {final_score} / 1.0")




🧠 Welcome to the Cognitive Check

1. What day of the week is it?
Your answer: Tuesday

2. Can you name three animals?
Your answer: Bird, Rabbit, Phone

3. Please repeat this sentence: The cat sat on the mat.
Your answer: The cat sat under the table

REMEMBER THIS NUMBER: 0087



















































Now type the number you saw: 0087

🧠 Word Recall Game
Memorize these words:
apple, table, car, banana, house



















































Now, please recall and type as many words as you remember (separated by commas):
Your answer: apple, table, house, car

🧾 Detailed Scoring Summary:

1. What day of the week is it?
Expected: Tuesday
Your Answer: Tuesday
Score: 1.0 / 1.0

2. Can you name three animals?
Expected: Any 3 animals
Your Answer: Bird, Rabbit, Phone
Score: 0.67 / 1.0

3. Please repeat this sentence: The cat sat on the mat.
Expected: The cat sat on the mat
Your Answer: The cat sat under the table
Score: 0.72 / 1.0

4. Number Recall G

In [ ]:
import csv
import pandas as pd
from datetime import datetime
import os
import pytz

kst = pytz.timezone("Asia/Seoul")

log_file = 'cognitive_log.csv'
write_header = not os.path.exists(log_file)


In [ ]:
nickname = input("🆔 Enter your nickname: ").strip()
phone_last4 = input("📱 Enter last 4 digits of your phone number: ").strip()
email = input("📧 Enter your email (optional): ").strip()


🆔 Enter your nickname: Dog
📱 Enter last 4 digits of your phone number: 8900
📧 Enter your email (optional): 


In [ ]:
user_id = f"{nickname}_{phone_last4}_{email if email else 'noemail'}"
timestamp = datetime.now(kst).strftime("%Y-%m-%d %H:%M:%S")



In [ ]:
with open(log_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    if write_header:
        writer.writerow(['user_id', 'timestamp', 'question_type', 'question', 'expected', 'answer', 'score'])

    for entry in score_details:
        writer.writerow([
            user_id,
            timestamp,
            entry['type'],
            entry['question'],
            entry['expected'],
            entry['user_answer'],
            entry['score']
        ])

print(f"\n✅ Session saved to {log_file}\n")

# Load full log and summarize
df = pd.read_csv(log_file)
user_df = df[df['user_id'] == user_id]

# Group by session (timestamp) and calculate average score
session_summary = user_df.groupby('timestamp')['score'].mean().reset_index()

# Text summary output
print(f"📊 Progress Summary for {user_id}")
for i, row in session_summary.iterrows():
    print(f"Session {i+1} - {row['timestamp']}: Avg Score = {round(row['score'], 2)}")


✅ Session saved to cognitive_log.csv

📊 Progress Summary for Dog_8900_noemail
Session 1 - 2025-05-06 16:17:38: Avg Score = 0.84


In [ ]:
from transformers import pipeline

# Load the log and filter current user
df = pd.read_csv('cognitive_log.csv')
user_df = df[df['user_id'] == user_id]

# Sort by timestamp
user_df = user_df.sort_values(by='timestamp')

# Only aggregate numeric column (score)
last_two = user_df.groupby('timestamp')['score'].mean().tail(2).reset_index()

if len(last_two) < 2:
    print("🧠 Not enough data to generate trend-based feedback. Complete at least two sessions.")
else:
    prev_score = round(last_two.iloc[0]['score'], 2)
    curr_score = round(last_two.iloc[1]['score'], 2)
    trend = "improved" if curr_score > prev_score else "declined" if curr_score < prev_score else "stayed consistent"

    # 🧠 Construct prompt for LLM feedback agent
    prompt = f"""
    The user {user_id} completed a cognitive test today.
    Previous session score: {prev_score}
    Current session score: {curr_score}
    Trend: {trend}

    Please write a brief feedback message (1-2 sentences) encouraging the user and suggesting a next cognitive task to try.
    """

    # Use local summarization model or GPT-like model (use your own LLM if needed)
    try:
        summarizer = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_length=100)
        response = summarizer(prompt)[0]['generated_text']
        print("\n🤖 Personalized Feedback Agent:\n" + response.strip())
    except:
        print(f"\n🤖 Feedback:\nYour cognitive score has {trend} from {prev_score} to {curr_score}. Keep going!")


🧠 Not enough data to generate trend-based feedback. Complete at least two sessions.


In [ ]:
try:
    df = pd.read_csv(log_file)
    user_df = df[df['user_id'] == user_id]
    latest_session = user_df.groupby('timestamp')['score'].mean().reset_index().tail(1)
    last_score = float(latest_session['score'].values[0])
    print(f"\n📊 Last session score: {last_score}")
except:
    last_score = None
    print("\n📊 No previous session found. Using default settings.")


📊 Last session score: 0.8380000000000001


Adjust Game Parameters Based on Last Score


In [ ]:
import pandas as pd

df = pd.read_csv("/content/cognitive_log.csv")
print("✅ File Loaded.")
print(f"🔢 Number of rows: {len(df)}")
print(f"📋 Columns: {df.columns.tolist()}")
print("\n🔍 Preview:")
print(df.head(10))

✅ File Loaded.
🔢 Number of rows: 20
📋 Columns: ['user_id', 'timestamp', 'question_type', 'question', 'expected', 'answer', 'score']

🔍 Preview:
            user_id            timestamp question_type  \
0  dog_8900_noemail  2025-05-05 07:35:47          text   
1  dog_8900_noemail  2025-05-05 07:35:47          text   
2  dog_8900_noemail  2025-05-05 07:35:47          text   
3  dog_8900_noemail  2025-05-05 07:35:47        memory   
4  dog_8900_noemail  2025-05-05 07:35:47        recall   
5  dog_8900_noemail  2025-05-05 07:50:19          text   
6  dog_8900_noemail  2025-05-05 07:50:19          text   
7  dog_8900_noemail  2025-05-05 07:50:19          text   
8  dog_8900_noemail  2025-05-05 07:50:19        memory   
9  dog_8900_noemail  2025-05-05 07:50:19        recall   

                                            question  \
0                     1. What day of the week is it?   
1                     2. Can you name three animals?   
2  3. Please repeat this sentence: The cat sat on

In [ ]:
import pandas as pd
from datetime import datetime
import pytz

# 🕒 Get Korean time (Asia/Seoul)
kst = pytz.timezone("Asia/Seoul")
timestamp = datetime.now(kst).strftime("%Y-%m-%d %H:%M:%S")

# ✅ User ID input
nickname = input("🆔 Enter your nickname: ").strip().lower()
phone_digits = input("📱 Enter last 4 digits of your phone number: ").strip()
user_prefix = f"{nickname}_{phone_digits}"

# ✅ Load log file
log_file = "cognitive_log.csv"
df = pd.read_csv(log_file)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# ✅ Match any user_id starting with this prefix
matching_ids = df[df['user_id'].str.startswith(user_prefix)]['user_id'].unique()

if len(matching_ids) == 0:
    print("📊 No previous sessions found for this user.")
    user_id = user_prefix  # For new session
    last_score = None
else:
    user_id = matching_ids[0]  # Use first match
    user_df = df[df['user_id'] == user_id]
    last_timestamp = user_df['timestamp'].max()

    # 🕒 Convert previous session time to Korean time
    last_time_kst = last_timestamp.tz_localize('UTC').astimezone(kst).strftime('%Y-%m-%d %H:%M:%S')

    last_score = user_df[user_df['timestamp'] == last_timestamp]['score'].mean()
    print(f"📊 Found previous sessions for {user_id} ({last_time_kst})")
    print(f"📈 Last Session Avg Score: {round(last_score, 2)}")


🆔 Enter your nickname: Dog
📱 Enter last 4 digits of your phone number: 8900
📊 Found previous sessions for dog_8900_noemail (2025-05-06 01:52:04)
📈 Last Session Avg Score: 0.85


LLM-driven adaptive assessment system

In [ ]:
# === Game 1: Adaptive Number Recall ===
def number_recall_game_adaptive():
    length = 5
    if last_score is not None:
        if last_score > 0.85:
            length = 7
        elif last_score < 0.6:
            length = 4

    number = ''.join(random.choices('0123456789', k=length))
    print(f"\n🔢 REMEMBER THIS NUMBER: {number}")
    time.sleep(3)
    print("\n" * 50)
    user_input = input("Now type the number you saw: ").strip()
    score = 1.0 if user_input == number else 0.0
    return score, number, user_input

memory_score, number_target, number_input = number_recall_game_adaptive()
score_details.append({
    "type": "memory",
    "question": "🔢 Number Recall",
    "expected": number_target,
    "user_answer": number_input,
    "score": memory_score
})

# === Game 2: Adaptive Word Recall ===
def word_list_recall_game_adaptive():
    easy = ["apple", "table", "car", "banana", "house"]
    hard = ["zebra", "guitar", "museum", "pyramid", "canvas"]
    word_list = easy
    if last_score and last_score > 0.85:
        word_list = hard

    print("\n🧠 Word Recall Game")
    print("Memorize these words:")
    print(", ".join(word_list))
    time.sleep(5)
    print("\n" * 50)

    user_input = input("Now, type as many words as you remember (comma-separated): ").lower()
    user_words = [w.strip() for w in user_input.split(",")]
    correct = sum([1 for word in word_list if word in user_words])
    score = round(correct / len(word_list), 2)
    return score, ", ".join(word_list), ", ".join(user_words)

recall_score, word_target, word_input = word_list_recall_game_adaptive()
score_details.append({
    "type": "recall",
    "question": "🧠 Word List Recall",
    "expected": word_target,
    "user_answer": word_input,
    "score": recall_score
})

# === Game 3: Adaptive Sentence Repetition ===
def sentence_repetition_game_adaptive():
    if last_score is not None:
        if last_score > 0.85:
            sentence = "Despite the complexity of the situation, the cat remained calm on the velvet cushion."
        elif last_score < 0.6:
            sentence = "The dog ran fast."
        else:
            sentence = "The cat sat on the mat."
    else:
        sentence = "The cat sat on the mat."

    print("\n3. Please repeat this sentence exactly:")
    print(sentence)
    user_input = input("Your answer: ")
    model = SentenceTransformer('all-MiniLM-L6-v2')
    score = util.cos_sim(model.encode(sentence, convert_to_tensor=True), model.encode(user_input, convert_to_tensor=True)).item()
    return round(score, 2), sentence, user_input

sentence_score, sentence_target, sentence_input = sentence_repetition_game_adaptive()
print(f"\n✅ Sentence Similarity Score: {sentence_score} / 1.0")
score_details.append({
    "type": "language",
    "question": "🗣 Sentence Repetition",
    "expected": sentence_target,
    "user_answer": sentence_input,
    "score": sentence_score
})

# === Save to Log ===
write_header = not pd.io.common.file_exists(log_file)
with open(log_file, mode='a', newline='', encoding='utf-8') as file:
    import csv
    writer = csv.writer(file)
    if write_header:
        writer.writerow(['user_id', 'timestamp', 'question_type', 'question', 'expected', 'answer', 'score'])
    for entry in score_details:
        writer.writerow([
            user_id,
            timestamp,
            entry['type'],
            entry['question'],
            entry['expected'],
            entry['user_answer'],
            entry['score']
        ])
print(f"\n✅ Session saved to {log_file}")

# === Comparison with Last Session + Feedback ===
df = pd.read_csv(log_file)
df['timestamp'] = pd.to_datetime(df['timestamp'])
user_df = df[df['user_id'] == user_id]
recent_sessions = user_df['timestamp'].drop_duplicates().sort_values().tail(2).tolist()

if len(recent_sessions) < 2:
    print("\n📊 Only one session available. Do more sessions to compare trends.")
else:
    last_time, current_time = recent_sessions[-2], recent_sessions[-1]
    last_session = user_df[user_df['timestamp'] == last_time]
    current_session = user_df[user_df['timestamp'] == current_time]

    last_avg = round(last_session['score'].mean(), 2)
    current_avg = round(current_session['score'].mean(), 2)
    avg_change = round(current_avg - last_avg, 2)
    avg_arrow = "↑" if avg_change > 0 else "↓" if avg_change < 0 else "→"

    last_task = last_session.groupby('question_type')['score'].mean()
    current_task = current_session.groupby('question_type')['score'].mean()
    all_tasks = sorted(set(current_task.index).union(set(last_task.index)))

    print(f"\n🧾 Session Comparison Summary:")
    print(f"\n📅 Previous Session: {last_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📈 Overall Score: {last_avg} / 1.0")

    print(f"\n📅 Current Session: {current_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📈 Overall Score: {current_avg} / 1.0 ({avg_arrow} {avg_change:+.2f})")

    print(f"\n📊 Task-wise Comparison:")
    for task in all_tasks:
        prev = round(last_task.get(task, 0.0), 2)
        curr = round(current_task.get(task, 0.0), 2)
        change = round(curr - prev, 2)
        arrow = "↑" if change > 0 else "↓" if change < 0 else "→"
        print(f"• {task.capitalize():<10} | {prev} → {curr} ({arrow} {change:+.2f})")



🔢 REMEMBER THIS NUMBER: 5949402



















































Now type the number you saw: 594902

🧠 Word Recall Game
Memorize these words:
zebra, guitar, museum, pyramid, canvas



















































Now, type as many words as you remember (comma-separated): zebara, guitar, museaum, others

3. Please repeat this sentence exactly:
Despite the complexity of the situation, the cat remained calm on the velvet cushion.
Your answer: Despite the complexity of the situation, the cat remained angry on the velvet cushion

✅ Sentence Similarity Score: 0.89 / 1.0

✅ Session saved to cognitive_log.csv

🧾 Session Comparison Summary:

📅 Previous Session: 2025-05-05 16:52:04
📈 Overall Score: 0.85 / 1.0

📅 Current Session: 2025-05-06 16:21:11
📈 Overall Score: 0.66 / 1.0 (↓ -0.19)

📊 Task-wise Comparison:
• Language   | 0.0 → 0.89 (↑ +0.89)
• Memory     | 1.0 → 0.5 (↓ -0.50)
• Recall     | 0.8 → 0.5 (↓ -0.30)
• Text       | 0.82 → 0.8 (↓ -0.02)


In [ ]:
# === LLM Smart Feedback ===
task_feedback_lines = []
for task in all_tasks:
    prev = round(last_task.get(task, 0.0), 2)
    curr = round(current_task.get(task, 0.0), 2)
    change = round(curr - prev, 2)
    arrow = "↑" if change > 0 else "↓" if change < 0 else "→"
    task_feedback_lines.append(f"{task.capitalize()}: {prev} → {curr} ({arrow} {change:+.2f})")

task_feedback_str = "\n".join(task_feedback_lines)

prompt = f"""
The user just completed two cognitive assessment sessions.
Compare these task scores:

{task_feedback_str}

Their overall score changed from {last_avg} to {current_avg} ({avg_arrow} {avg_change:+.2f}).

Write a 2-3 sentence feedback message that:
- Highlights an area of improvement or decline
- Suggests one simple cognitive task for tomorrow
- Keeps the tone positive and helpful
"""

try:
    summarizer = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_length=150)
    response = summarizer(prompt)[0]['generated_text']
    print("\n🤖 Personalized Feedback Agent:\n" + response.strip())
except:
    weak_task = min(all_tasks, key=lambda t: current_task.get(t, 1.0))
    strong_task = max(all_tasks, key=lambda t: current_task.get(t, 0.0))

    print("\n🤖 Personalized Feedback Agent:")
    print(f"Great job on your {strong_task} task — you're maintaining strong performance! 💡")
    print(f"Consider spending a few minutes tomorrow practicing a simple {weak_task} task, like repeating numbers or word recall.")
    print("You're making steady progress. Keep it up! 💪")


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🤖 Personalized Feedback Agent:
Great job on your language task — you're maintaining strong performance! 💡
Consider spending a few minutes tomorrow practicing a simple memory task, like repeating numbers or word recall.
You're making steady progress. Keep it up! 💪


If you want this working without external API or Hugging Face, we can:
Use a fine-tuned local model (like flan-t5 or tinyllama)


Cognitive UI App


In [ ]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import random
import pandas as pd
from datetime import datetime
import pytz
from sentence_transformers import SentenceTransformer, util
import torch

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Get KST time
def get_korean_time():
    kst = pytz.timezone("Asia/Seoul")
    return datetime.now(kst).strftime("%Y-%m-%d %H:%M:%S")

# Load log file
def load_log():
    try:
        df = pd.read_csv("cognitive_log.csv")
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df
    except:
        return pd.DataFrame(columns=['user_id', 'timestamp', 'question_type', 'question', 'expected', 'answer', 'score'])

# Save session to log
def save_session(user_id, timestamp, score_details):
    df = load_log()
    new_data = pd.DataFrame(score_details)
    new_data['user_id'] = user_id
    new_data['timestamp'] = timestamp
    df = pd.concat([df, new_data], ignore_index=True)
    df.to_csv("cognitive_log.csv", index=False)

# Get last task scores
def get_last_task_scores(df, user_id):
    user_df = df[df['user_id'] == user_id]
    if user_df.empty:
        return {}, None
    last_time = user_df['timestamp'].max()
    last_session = user_df[user_df['timestamp'] == last_time]
    return last_session.groupby('question_type')['score'].mean().to_dict(), last_time

# Step functions
def step_nickname(nickname, phone):
    user_id = f"{nickname.strip().lower()}_{phone.strip()}"
    df_log = load_log()
    last_scores, last_time = get_last_task_scores(df_log, user_id)
    puzzle, expected, explain = step_sudoku()
    return gr.update(visible=True), user_id, last_scores, gr.update(value=puzzle, visible=True), expected, explain, gr.update(visible=True)

# === Sudoku Puzzle Game ===
def generate_sudoku_puzzle():
    numbers = list(range(1, 6))
    solution = random.sample(numbers, len(numbers))
    missing_index = random.randint(0, len(solution) - 1)
    row = solution.copy()
    row[missing_index] = None
    display_row = " ".join(str(x) if x is not None else "?" for x in row)
    explanation = f"The missing number is {solution[missing_index]} because it completes the sequence without repetition."
    return display_row, str(solution[missing_index]), explanation

def step_sudoku():
    return generate_sudoku_puzzle()

def score_sudoku_guess(user_input, expected):
    if user_input is None or expected is None:
        return 0.0
    return 1.0 if user_input.strip() == expected.strip() else 0.0

def step_sentence_recall(user_id):
    lang_last = get_last_task_scores(load_log(), user_id)[0].get("language", 0.7)
    sentence = "Despite the complexity of the situation, the cat remained calm on the velvet cushion." if lang_last > 0.85 else "The dog ran fast." if lang_last < 0.6 else "The cat sat on the mat."
    return sentence, sentence

def step_submit(sentence_input, sentence_target_state, user_id, sudoku_input, sudoku_expected_state, sudoku_explanation):
    sentence_target = sentence_target_state or ""
    sudoku_expected = sudoku_expected_state or ""

    if not all([sentence_input, sentence_target, user_id, sudoku_input, sudoku_expected]):
        return "❌ Error: One or more inputs are missing. Please make sure to complete all tasks before submitting."

    try:
        sim_score = util.cos_sim(
            model.encode(sentence_target, convert_to_tensor=True),
            model.encode(sentence_input, convert_to_tensor=True)
        ).item()
        lang_score = round(sim_score, 2)
        sudoku_score = score_sudoku_guess(sudoku_input, sudoku_expected)

        timestamp = get_korean_time()
        score_details = [
            {"question_type": "sudoku", "question": "Sudoku Guess", "expected": sudoku_expected, "answer": sudoku_input, "score": sudoku_score},
            {"question_type": "language", "question": "Sentence Repetition", "expected": sentence_target, "answer": sentence_input, "score": lang_score},
        ]
        save_session(user_id, timestamp, score_details)

        feedback = f"✅ Session saved for {user_id} at {timestamp}\n"
        for item in score_details:
            feedback += f"{item['question_type'].capitalize()}: {item['score']} / 1.0\n"
            feedback += f"  ↳ Expected: {item['expected']}\n"
            feedback += f"  ↳ Your Answer: {item['answer']}\n"
        if sudoku_score < 1.0:
            feedback += f"💡 Sudoku Explanation: {sudoku_explanation}\n"
        return feedback
    except Exception as e:
        return f"❌ Error occurred: {str(e)}"

with gr.Blocks(theme=gr.themes.Base(primary_hue="orange")) as demo:
    gr.Markdown("## 🧠 Early Dementia Cognitive Check (Multistep)")

    with gr.Row():
        nickname = gr.Textbox(label="Nickname")
        phone = gr.Textbox(label="Last 4 digits of Phone")
        user_id = gr.State()
        last_scores = gr.State()
        submit_btn = gr.Button("Start Assessment")

    with gr.Group(visible=False) as step1:
        sudoku_display = gr.Textbox(label="Sudoku Row (guess missing number)", interactive=False)
        sudoku_expected = gr.State()
        sudoku_explanation = gr.State()
        sudoku_input = gr.Textbox(label="What is the missing number ( ? )")
        sudoku_btn = gr.Button("Next")

    with gr.Group(visible=False) as step2:
        sentence_display = gr.Textbox(label="Repeat this sentence", interactive=False)
        sentence_target = gr.State()
        sentence_input = gr.Textbox(label="Your response")
        result = gr.Textbox(label="Session Result", lines=6)
        final_btn = gr.Button("Submit")

    submit_btn.click(step_nickname, inputs=[nickname, phone], outputs=[step1, user_id, last_scores, sudoku_display, sudoku_expected, sudoku_explanation, sudoku_btn])
    sudoku_btn.click(lambda: gr.update(visible=True), None, step2)
    sudoku_btn.click(step_sentence_recall, inputs=[user_id], outputs=[sentence_display, sentence_target])
    final_btn.click(step_submit, inputs=[sentence_input, sentence_target, user_id, sudoku_input, sudoku_expected, sudoku_explanation], outputs=result)


demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0fcf6eda846525b408.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Added Image

In [ ]:
import gradio as gr
import random
import pandas as pd
from datetime import datetime
import pytz
from sentence_transformers import SentenceTransformer, util
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import io

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Get KST time
def get_korean_time():
    kst = pytz.timezone("Asia/Seoul")
    return datetime.now(kst).strftime("%Y-%m-%d %H:%M:%S")

# Load log file
def load_log():
    try:
        df = pd.read_csv("cognitive_log.csv")
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df
    except:
        return pd.DataFrame(columns=['user_id', 'timestamp', 'question_type', 'question', 'expected', 'answer', 'score'])

# Save session to log
def save_session(user_id, timestamp, score_details):
    df = load_log()
    new_data = pd.DataFrame(score_details)
    new_data['user_id'] = user_id
    new_data['timestamp'] = timestamp
    df = pd.concat([df, new_data], ignore_index=True)
    df.to_csv("cognitive_log.csv", index=False)

# Get last task scores
def get_last_task_scores(df, user_id):
    user_df = df[df['user_id'] == user_id]
    if user_df.empty:
        return {}, None
    last_time = user_df['timestamp'].max()
    last_session = user_df[user_df['timestamp'] == last_time]
    return last_session.groupby('question_type')['score'].mean().to_dict(), last_time

# === Bingo Puzzle Game ===
def draw_shape(ax, shape, x, y):
    if shape == 'circle':
        ax.add_patch(patches.Circle((x + 0.5, y + 0.5), 0.35, color='skyblue'))
    elif shape == 'square':
        ax.add_patch(patches.Rectangle((x + 0.1, y + 0.1), 0.8, 0.8, color='lightgreen'))
    elif shape == 'triangle':
        triangle = patches.Polygon([[x + 0.5, y + 0.9], [x + 0.1, y + 0.1], [x + 0.9, y + 0.1]], color='lightcoral')
        ax.add_patch(triangle)
    elif shape == 'question':
        ax.text(x + 0.5, y + 0.5, '?', fontsize=30, ha='center', va='center')

def render_bingo_image(grid):
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.set_xlim(0, 3)
    ax.set_ylim(0, 3)
    ax.axis('off')
    for y in range(3):
        for x in range(3):
            draw_shape(ax, grid[y][x], x, 2 - y)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    return np.array(Image.open(buf))  # <- convert to array for Gradio


def render_bingo_image(grid):
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.set_xlim(0, 3)
    ax.set_ylim(0, 3)
    ax.axis('off')
    for y in range(3):
        for x in range(3):
            draw_shape(ax, grid[y][x], x, 2 - y)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)

# Gradio Interface
def start_bingo_game():
    grid, expected, pos = generate_bingo_puzzle()
    img = render_bingo_image(grid)
    return img, expected

def check_bingo_answer(user_input, expected):
    if user_input.strip().lower() == expected:
        return "✅ Correct!"
    else:
        return f"❌ Wrong. The correct answer was: {expected}"

with gr.Blocks() as demo:
    gr.Markdown("## 🎯 Bingo Visual Puzzle Game")

    image_output = gr.Image(label="Bingo Grid")
    hidden_expected = gr.Textbox(visible=False)

    with gr.Column():
        user_input = gr.Textbox(label="🤔 What shape is missing? (circle/square/triangle)")
        result_output = gr.Textbox(label="Result")

    start_btn = gr.Button("Start Game")
    submit_btn = gr.Button("Submit")

    start_btn.click(start_bingo_game, inputs=[], outputs=[image_output, hidden_expected])
    submit_btn.click(check_bingo_answer, inputs=[user_input, hidden_expected], outputs=result_output)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8ac04b9292df3c32f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install pytesseract
!apt-get install -y tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
print(f"📊 Progress Summary for {user_id}")
for i, row in session_summary.iterrows():
    print(f"Session {i+1} - {row['timestamp']}: Avg Score = {round(row['score'], 2)}")

📊 Progress Summary for <gradio.components.state.State object at 0x7e86c10345d0>
Session 1 - 2025-05-06 16:17:38: Avg Score = 0.84


In [ ]:
import gradio as gr
import pandas as pd
from datetime import datetime
import pytz
from PIL import Image
import pytesseract
import re

# Get KST time
def get_korean_time():
    kst = pytz.timezone("Asia/Seoul")
    return datetime.now(kst).strftime("%Y-%m-%d %H:%M:%S")

# Load log file
def load_log():
    try:
        df = pd.read_csv("cognitive_log.csv")
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df
    except:
        return pd.DataFrame(columns=['user_id', 'timestamp', 'question_type', 'question', 'expected', 'answer', 'score'])

# Save session to log
def save_session(user_id, timestamp, score_details):
    df = load_log()
    new_data = pd.DataFrame(score_details)
    new_data['user_id'] = user_id
    new_data['timestamp'] = timestamp
    df = pd.concat([df, new_data], ignore_index=True)
    df.to_csv("cognitive_log.csv", index=False)

# Extract text from uploaded image using OCR
def extract_text_from_image(image_path):
    try:
        img = Image.open(image_path)
        text = pytesseract.image_to_string(img)
        return text
    except Exception as e:
        return f"OCR failed: {str(e)}"

# Extract fields from OCR text
def extract_fields_from_text(text):
    fields = {
        "nickname": None,
        "digits": None,
        "game_name": None,
        "score": None,
        "date": None,
        "agent": None
    }

    patterns = {
        "nickname": r"Nickname:\s*(\w+)",
        "digits": r"Last 4 digits.*?:\s*(\d{4})",
        "game_name": r"Game name:\s*(.+)",
        "score": r"Score:\s*(\d+)",
        "date": r"Date:\s*(\d+)",
        "agent": r"Agent:\s*(\w+)"
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            fields[key] = match.group(1).strip()

    return fields

# Compare score to previous 2 sessions

def compare_performance(user_id, game_name, current_score):
    df = load_log()
    user_df = df[(df['user_id'] == user_id) & (df['question_type'] == game_name)]
    if user_df.empty:
        return "📊 No previous record found for comparison."

    user_df = user_df.sort_values('timestamp', ascending=False).head(3)
    rows = []
    for i, row in user_df.iterrows():
        rows.append(f"{row['timestamp'].strftime('%Y-%m-%d')} | {row['question_type']} | Score: {row['score']}")

    if len(rows) >= 2:
        diff = float(user_df.iloc[0]['score']) - float(user_df.iloc[1]['score'])
        trend = "⬆️ Improved" if diff > 0 else ("⬇️ Declined" if diff < 0 else "➡️ Same")
        feedback = f"Recent Scores:\n" + "\n".join(rows) + f"\nTrend: {trend} ({diff:+.1f})"
    else:
        feedback = f"Only one session found:\n" + "\n".join(rows)

    return feedback

# Upload-based offline entry
def process_uploaded_image(image_path, user_id, digits, game_name, score):
    timestamp = get_korean_time()
    ocr_text = extract_text_from_image(image_path)
    fields = extract_fields_from_text(ocr_text)

    # Fill missing inputs with OCR data
    nickname = user_id or fields['nickname']
    digits = digits or fields['digits']
    game_name = game_name or fields['game_name']
    score = score or fields['score']

    if nickname and digits and game_name and score:
        user_id_combined = f"{nickname.strip().lower()}_{digits.strip()}"
        result = {
            "user_id": user_id_combined,
            "timestamp": timestamp,
            "question_type": game_name,
            "question": "offline_upload",
            "expected": "N/A",
            "answer": f"OCR via {fields.get('agent', 'unknown')}",
            "score": float(score)
        }
        save_session(user_id_combined, timestamp, [result])

        performance_feedback = compare_performance(user_id_combined, game_name, score)

        return f"✅ Saved result for {nickname} ({game_name}) score: {score}\n🧠 Agent: {fields.get('agent', 'N/A')}\n📅 Date: {fields.get('date', 'N/A')}\n\n{performance_feedback}\n\n🖼️ OCR Text:\n{ocr_text}"
    else:
        return f"⚠️ Could not extract all fields.\n🖼️ OCR Text:\n{ocr_text}"

with gr.Blocks() as demo:
    gr.Markdown("## 🖨️ Upload Offline Assessment (Printout)")
    with gr.Row():
        upload_img = gr.Image(type="filepath", label="Upload Completed Puzzle")
        id_input = gr.Textbox(label="Nickname (Optional)")
        digit_input = gr.Textbox(label="Last 4 digits (Optional)")
        game_name = gr.Textbox(label="Game name (Optional)")
        score_input = gr.Textbox(label="Score (Optional)")
    upload_result = gr.Textbox(label="Result / OCR Output", lines=6)
    upload_btn = gr.Button("Upload and Process")

    upload_btn.click(
        process_uploaded_image,
        inputs=[upload_img, id_input, digit_input, game_name, score_input],
        outputs=upload_result
    )

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eeb7dd03d22fd61381.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


"A Multimodal Interactive-Agent Framework for Inclusive and Personalized Early Dementia Assessment Using Online and Offline Cognitive Tasks"

Novelty & Contributions:
Multimodal Assessment Integration

Combines online cognitive agents (e.g., adaptive language/sudoku games) with offline printable puzzles completed by seniors and later uploaded via scanned images.

Uses OCR (Tesseract) to extract user identity, task type, and scores from printouts, enabling consistent tracking across modalities.

Personalized Difficulty Adaptation with LLMs

Tracks past task-specific scores (e.g., language, sudoku, recall) to dynamically adjust game difficulty.

Uses a lightweight transformer (MiniLM) to evaluate semantic similarity and sentence recall precision.

Applies LLM-based feedback generation (e.g., Falcon-RW, FLAN-T5) to deliver natural language encouragement, recommendations, and explanations.

Offline + Online Logging in Unified Session Tracker

All sessions—whether interactive online or OCR-scanned offline—are stored in a structured log (CSV/DB), including timestamps, scores, task types, and source.

Enables longitudinal analysis for performance trends and deterioration/improvement flags.

Accessible for Low Digital Literacy Populations

Offline printout support allows digitally underserved elderly to participate via family or caregivers.

Hybrid approach improves inclusivity in healthcare tech adoption.

Real-time Visual and Performance Feedback

Real-time charting or tabular output compares latest vs previous session scores.

Tracks improvement trends by task type (e.g., "Your language performance improved by 0.2 since last week").